In [6]:
from langchain_groq import ChatGroq
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
import sqlite3
from sqlalchemy import create_engine
from pathlib import Path


In [2]:
api_key = "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"

In [3]:
llm = ChatGroq(api_key=api_key, model_name="llama-3.1-70b-versatile", streaming=True)

In [4]:
import sqlite3

# Create database and tables
conn = sqlite3.connect("database.db")
cursor = conn.cursor()

# Create assignments table
cursor.execute("""
CREATE TABLE IF NOT EXISTS assignments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject TEXT,
    deadline DATE,
    question TEXT
)
""")

# Create attendance table
cursor.execute("""
CREATE TABLE IF NOT EXISTS attendance (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT,
    enrollment TEXT,
    attendance_percentage FLOAT
)
""")

# Insert sample data
cursor.execute("INSERT INTO assignments (subject, deadline, question) VALUES ('DBMS', '2024-10-25', 'Explain normalization.')")
cursor.execute("INSERT INTO attendance (student_name, enrollment, attendance_percentage) VALUES ('John Doe', 'EN1234', 85.5)")

conn.commit()
cursor.close()
conn.close()

print("Database initialized successfully.")


Database initialized successfully.


In [1]:
import sqlite3

conn = sqlite3.connect("database.db")
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS assignments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject TEXT NOT NULL,
    deadline TEXT NOT NULL,
    question TEXT NOT NULL
)
''')

conn.commit()
conn.close()


In [11]:
db_path = (Path("database.db")).absolute()
print(db_path)
creator = lambda: sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
db =  SQLDatabase(create_engine("sqlite:///",creator = creator))

c:\code\MinorProject@KPK\New folder\database.db


In [12]:
toolkit = SQLDatabaseToolkit(db=db, llm = llm)
agent = create_sql_agent(
    llm = llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,handle_parsing_errors=True
    )

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\agent_toolkits\sql\base.py:138: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [13]:
User_query = "What is the attendance percentage of john doe?"

In [15]:
response = agent.run(User_query)



> Entering new SQL Agent Executor chain...
Thought: To find the attendance percentage of John Doe, I first need to know what tables exist in the database that may contain information about student attendance. 

Action: sql_db_list_tables 
Action Input: assignments, attendanceThought: Now that I know the tables in the database, I can see that there is an 'attendance' table which is likely where I will find the information I need. However, I'm not sure what fields this table contains, so I should check the schema of the 'attendance' table.

Action: sql_db_schema
Action Input: attendance
CREATE TABLE attendance (
	id INTEGER, 
	student_name TEXT, 
	enrollment TEXT, 
	attendance_percentage FLOAT, 
	PRIMARY KEY (id)
)

/*
3 rows from attendance table:
id	student_name	enrollment	attendance_percentage
1	John Doe	EN1234	85.5
*/Thought: Now that I know the schema of the 'attendance' table, I can see that the table contains a 'student_name' field and an 'attendance_percentage' field. I can use